#Image corpus creation and Transfer Learning in CNN

1. Import the necessary libraries

In [ ]:
import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten
from keras.layers import Conv2D, MaxPooling2D

2. Initialize some parameters

In [ ]:
now = datetime.datetime.now
batch_size = 128
num_classes = 5
epochs = 5
img_rows, img_cols = 28,28
filters = 32
pool_size = 2
kernel_size = 3

3. Partition MNIST dataset

In [ ]:
(x_train,y_train),(x_test,y_test) = mnist.load_data()

In [ ]:
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]
x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5]-5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5]-5

4.Define the "feature" layers

In [ ]:
input_shape = (img_rows,img_cols,1)

In [ ]:
feature_layers=([Conv2D(filters=filters,kernel_size = kernel_size,activation='relu',input_shape=(img_rows,img_cols,1)),
                Conv2D(filters=filters,kernel_size = kernel_size,activation='relu',input_shape=(img_rows,img_cols,1)),
                MaxPooling2D(pool_size=pool_size),
                Dropout(0.25),
                Flatten()])

5. Define the "classification" layers

In [ ]:
classification_layers = [Dense(128,activation='relu'),
Dropout(0.5),

Dense(5,activation='softmax')]

6.Define  a Sequential model

In [ ]:
model1 = Sequential(feature_layers+classification_layers)
model1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 conv2d_9 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 12, 12, 32)        0         
                                                                 
 flatten_2 (Flatten)         (None, 4608)              0         
                                                                 
 dense_4 (Dense)             (None, 128)               589952    
                                                      

7.Create a function train model()

In [ ]:
def train_model1(model1,train,test,num_classes):
  train[0] = train[0].reshape(train[0].shape[0],img_rows,img_cols,1)
  test[0] = test[0].reshape(test[0].shape[0],img_rows,img_cols,1)
  train[0] = train[0].astype('float32')
  test[0] = test[0].astype('float32')
  train[0] /= 255
  test[0] /= 255
  print(train[0].shape)
  print(test[0].shape)
  s = pd.Series(train[1])
  train[1] = pd.get_dummies(s)
  train[1] = train[1].values
  s = pd.Series(test[1])
  test[1] = pd.get_dummies(s)
  test[1] = test[1].values

  model1.compile(optimizer='adadelta',loss='categorical_crossentropy',metrics=['accuracy']) 

  train_start_time=now()

  model1.fit(train[0],train[1],epochs=epochs,verbose=2)

  print('Training time: %s' % (now() - train_start_time))

  score=model1.evaluate(test[0],test[1],verbose=0)

  print('test loss ',score[0])
  print('test accuracy ',score[1])

8.Training on digits 5 to 9

In [ ]:
train_model1(model1,[x_train_gte5,y_train_gte5],
             [x_test_gte5,y_test_gte5],num_classes)

(29404, 28, 28, 1)
(4861, 28, 28, 1)
Epoch 1/5
919/919 - 50s - loss: 1.5781 - accuracy: 0.2779 - 50s/epoch - 54ms/step
Epoch 2/5
919/919 - 54s - loss: 1.5167 - accuracy: 0.4302 - 54s/epoch - 58ms/step
Epoch 3/5
919/919 - 49s - loss: 1.4374 - accuracy: 0.5566 - 49s/epoch - 53ms/step
Epoch 4/5
919/919 - 49s - loss: 1.3367 - accuracy: 0.6327 - 49s/epoch - 54ms/step
Epoch 5/5
919/919 - 50s - loss: 1.2131 - accuracy: 0.6890 - 50s/epoch - 54ms/step
Training time: 0:04:11.735450
test loss  1.0883747339248657
test accuracy  0.8568195700645447


In [ ]:
import pandas as pd

9.Freeze Feature Layers

In [ ]:
for l in feature_layers:
  l.trainable = False

10. Print Summary

In [ ]:
model1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 conv2d_9 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 12, 12, 32)        0         
                                                                 
 flatten_2 (Flatten)         (None, 4608)              0         
                                                                 
 dense_4 (Dense)             (None, 128)               589952    
                                                      

11.Training for digits 0 to 4 based on digits 5 to 9

In [ ]:
train_model1(model1,[x_train_lt5, y_train_lt5],[x_test_lt5, y_test_lt5], num_classes)

(30596, 28, 28, 1)
(5139, 28, 28, 1)
Epoch 1/5
957/957 - 15s - loss: 1.5009 - accuracy: 0.3676 - 15s/epoch - 16ms/step
Epoch 2/5
957/957 - 15s - loss: 1.3133 - accuracy: 0.5306 - 15s/epoch - 16ms/step
Epoch 3/5
957/957 - 17s - loss: 1.1583 - accuracy: 0.6772 - 17s/epoch - 18ms/step
Epoch 4/5
957/957 - 16s - loss: 1.0254 - accuracy: 0.7639 - 16s/epoch - 17ms/step
Epoch 5/5
957/957 - 16s - loss: 0.9110 - accuracy: 0.8132 - 16s/epoch - 16ms/step
Training time: 0:01:19.022715
test loss  0.7852197289466858
test accuracy  0.9038723707199097


12. Reversing the training process

In [ ]:
feature_layers=([Conv2D(filters=filters,kernel_size = kernel_size,activation='relu',input_shape=(img_rows,img_cols,1)),
                Conv2D(filters=filters,kernel_size = kernel_size,activation='relu',input_shape=(img_rows,img_cols,1)),
                MaxPooling2D(pool_size=pool_size),
                Dropout(0.25),
                Flatten()])

In [ ]:
modelReverse = Sequential(feature_layers+classification_layers)
modelReverse.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 conv2d_11 (Conv2D)          (None, 24, 24, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 12, 12, 32)        0         
                                                                 
 flatten_3 (Flatten)         (None, 4608)              0         
                                                                 
 dense_4 (Dense)             (None, 128)               589952    
                                                      

In [ ]:
train_model1(modelReverse,[x_train_lt5,y_train_lt5],[x_test_lt5,y_test_lt5],num_classes)

(30596, 28, 28, 1)
(5139, 28, 28, 1)
Epoch 1/5
957/957 - 51s - loss: 1.2687 - accuracy: 0.7119 - 51s/epoch - 54ms/step
Epoch 2/5
957/957 - 50s - loss: 1.0312 - accuracy: 0.7904 - 50s/epoch - 53ms/step
Epoch 3/5
957/957 - 49s - loss: 0.7930 - accuracy: 0.8330 - 49s/epoch - 51ms/step
Epoch 4/5
957/957 - 49s - loss: 0.6092 - accuracy: 0.8597 - 49s/epoch - 51ms/step
Epoch 5/5
957/957 - 50s - loss: 0.4939 - accuracy: 0.8725 - 50s/epoch - 52ms/step
Training time: 0:04:09.514456
test loss  0.334087073802948
test accuracy  0.9340338706970215


In [ ]:
for l in feature_layers:
 l.trainable = False

In [ ]:
modelReverse.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 conv2d_11 (Conv2D)          (None, 24, 24, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 12, 12, 32)        0         
                                                                 
 flatten_3 (Flatten)         (None, 4608)              0         
                                                                 
 dense_4 (Dense)             (None, 128)               589952    
                                                      

In [ ]:
train_model1(modelReverse,[x_train_gte5,y_train_gte5],[x_test_gte5,y_test_gte5],num_classes)

(29404, 28, 28, 1)
(4861, 28, 28, 1)
Epoch 1/5
919/919 - 14s - loss: 1.5530 - accuracy: 0.4166 - 14s/epoch - 15ms/step
Epoch 2/5
919/919 - 13s - loss: 1.2671 - accuracy: 0.5017 - 13s/epoch - 14ms/step
Epoch 3/5
919/919 - 14s - loss: 1.0741 - accuracy: 0.5941 - 14s/epoch - 15ms/step
Epoch 4/5
919/919 - 14s - loss: 0.9389 - accuracy: 0.6624 - 14s/epoch - 15ms/step
Epoch 5/5
919/919 - 17s - loss: 0.8408 - accuracy: 0.7110 - 17s/epoch - 18ms/step
Training time: 0:01:11.992456
test loss  0.6890597939491272
test accuracy  0.7938695549964905


CASE 1 : Digits 5-9 is trained then last layer of network trained on
Digit 0-4

Digit 5-9 :
Training time: 0:04:11.735450  
test loss  1.0883747339248657     
test accuracy  0.8568195700645447   

Digit 0-4 :Training time: 0:01:19.022715       
test loss  0.7852197289466858      
test accuracy  0.9038723707199097     

CASE 2 : Digits 0-4 is trained then last layer of network trained on
Digit 5-9

Digit 0-4: Training time: 0:04:09.514456      
test loss  0.334087073802948       
test accuracy  0.9340338706970215       

Digit 5-9:Training time: 0:01:11.992456     
test loss  0.6890597939491272            
test accuracy  0.7938695549964905      